# 1. Loading the PETase dataset 

# 2. Loading external stability and expression datasets

# 3. The activity label of the PETase dataset 

**Docking** 

**PET catalysis rate** 

**PET-specific biophysical features** 

**Computational chemistry of PET and using it as a feature**

# 4. Fine-tuning esm2 model

# 5. Fine Tuning esm3 model

# 6. Graph Neural Network